In [5]:
import pandas as pd
import glob
import numpy as np
import shutil

path = 'C:\\ORS\\Data\\aggregated_sales_final.xlsx'
dest = 'C:\\ORS\\Data\\aggregated_sales_final1.xlsx'

data = pd.read_excel(path)
mask = mask = ~data['prodcode'].str.endswith(('*.jpg'))
data = data[mask]
data.to_excel(dest, index=False)